In [3]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
import pandas as pd
from collections import defaultdict
from libindic import inexactsearch
import pandas as pd
import pickle as pkl
import wptools
sys.path.append('..')
from method2.Places_translator import Places_translator
translator = Places_translator()
from method1.utils import *
from baseline.baseline import baseline_infobox
from method1.get_infobox import method1_infobox

In [10]:
def get_score(translated_infobox , actual_infobox):
    inst = inexactsearch.InexactSearch()
    markings = defaultdict(list)
    for key in translated_infobox.keys():
        if key in actual_infobox.keys() and key!='image':
#             print(key)
#             print(translated_infobox[key] , actual_infobox[key]) 
            val = inst.compare(translated_infobox[key] , actual_infobox[key])
            if val > 0.70 :
                markings['C'].append(key)
            else : markings['S'].append(key)
    for key in actual_infobox.keys():
        if key not in translated_infobox.keys():
            markings['D'].append(key)
    if len(markings['C']) + len(markings['S']) != 0:
        precision = len(markings['C']) / (len(markings['C']) + len(markings['S']))
    else : precision = 0
    recall = len(markings['C']) / (len(markings['C']) + len(markings['S']) + len(markings['D']))
    print('Precisions :',precision)
    print('Recall :',recall)
    return precision , recall

In [15]:
with open('../data-collection/places/eval_places.json') as f:
    data = json.loads(f.read())['data']
wiki_places = []
for entry in data:
    try :
        wiki_places.append([entry['en_wikipedia_title'] , entry['hi_wikipedia_title'] , entry['wd_id']])
    except :
        pass

In [19]:
def parse_infobox(text):
    for key in  ["{{Infobox","{{ज्ञानसन्दूक","{{Geobox","{{ज्ञानसंदूक"]:
        if key in text :
            text =  text.split(key)[1]
            break
    else: 
        return None
    text = text.split("\n")[1:]
    ret = {}
    for line in text:
        if line == "}}" : break
        if line[0] != '|' : continue
        line = line[2:].split(" = ")
        if len(line) == 1:
            x , y = line[0] , ""
        else:
            x , y = line[0] , line[1]
        ret[x] = y
    else :
        print("Parsing Error")
        assert(False)
    return ret

In [23]:
final_score = []
# 6 for example
for en_name , hi_name , qid in wiki_places:
    try:
        print(en_name , hi_name , qid)
        actual_infobox = parse_infobox(translator.get_page(hi_name , language = 'hi').data['wikitext'])
        if actual_infobox == None: continue
        baseline = baseline_infobox(qid , category= 'places')
        method2 = translator.get_infobox(en_name , test=True)
        method1 = method1_infobox(qid,'places')
        final_score.append({en_name : [method1 , method2 , baseline , actual_infobox]})
        with open('people_eval.jsonl' , 'a+') as f:
            f.write(json.dumps(final_score[-1] , ensure_ascii = False))
            f.write("\n")
    except Exception as e:
        print(e)
        print(en_name)
print()

India भारत Q668


hi.wikipedia.org (parse) भारत
भारत (hi) data
{
  infobox: <dict(6)> title, titlebar, left1, right1, float, bars
  iwlinks: <list(11)> https://en.wikipedia.org/wiki/Cambridge_Univ...
  pageid: 59
  parsetree: <str(121706)> <root><template><title>ज्ञानसन्दूक देश<...
  requests: <list(1)> parse
  title: भारत
  wikibase: Q668
  wikidata_url: https://www.wikidata.org/wiki/Q668
  wikitext: <str(97408)> {{ज्ञानसन्दूक देश| native_name = भारत गणर...
}
en.wikipedia.org (parse) India
India (en) data
{
  infobox: <dict(93)> conventional_long_name, common_name, native_...
  iwlinks: <list(9)> https://commons.wikimedia.org/wiki/Atlas_of_I...
  pageid: 14533
  parsetree: <str(498981)> <root><template><title>about</title><pa...
  requests: <list(1)> parse
  title: India
  wikibase: Q668
  wikidata_url: https://www.wikidata.org/wiki/Q668
  wikitext: <str(280342)> {{about|the Republic of India}}{{redirec...
}


In [29]:
import pickle as pkl
with open("final_places.pkl" , "wb") as f:
    pkl.dump(final_score , f)